In [1]:
import mne
from scipy import fftpack
from scipy.fft import fft
from scipy.signal import resample
from skimage import util
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
stage_names ={1.0:'awake',
              2.0:'PS',
              3.0:'SWS',
              10.0:"Artifact"}

In [16]:
raw_data = mne.io.read_raw_edf('data/CRFS60_1wk_withSleepStages.edf')

Extracting EDF parameters from D:\sleepscoring\data\CRFS60_1wk_withSleepStages.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [17]:
raw_data.info

<Info | 7 non-empty values
 bads: []
 ch_names: Activity, EEG, EMG, SignalStr, Temp, Signal-Sleep
 chs: 6 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: 2020-02-18 10:00:00 UTC
 nchan: 6
 projs: []
 sfreq: 500.0 Hz
>

In [18]:
df = raw_data.to_data_frame(['Activity', 'EEG', 'EMG', 'Signal-Sleep'])

In [19]:
df['Signal-Sleep']=df['Signal-Sleep']/1e6

In [20]:
df['Signal-Sleep']=np.abs(df['Signal-Sleep']).round()

In [21]:
df['Signal-Sleep'].value_counts()

1.0     20850000
3.0     19220000
2.0      2335000
10.0      795000
Name: Signal-Sleep, dtype: int64

In [29]:
sleep_stages = []
for i in range(8640):
    sleep_stages.append(df["Signal-Sleep"][i*5000:(i+1)*5000].mean())

In [35]:
data_dict =  {'time':[],
              'eeg':[],
              'emg':[],
              'signal_str':[],
              'sleep_stage':[]}


for i in range(6*60*24):
    data_dict['time'].append(df.time[i*5000])
    data_dict['eeg'].append(df.EEG[i*5000:(i+1)*5000].values)
    data_dict['emg'].append(df.EMG[i*5000:(i+1)*5000].values)
    data_dict['signal_str'].append(df.Activity[i*5000:(i+1)*5000].values)
    data_dict['sleep_stage'].append(df["Signal-Sleep"][i*5000])

In [36]:
df2 = pd.DataFrame.from_dict(data_dict)

In [37]:
df2.head()

,time,eeg,emg,signal_str,sleep_stage
0,0,"[22.125919370097964, 5.645924253059481, -6.408...","[3.662221137119663, 4.119998779259621, 5.03555...","[1953.1846064638203, 1953.1846064638203, 1953....",1.0
1,10000,"[-29.755546739097262, -16.93777275917844, -3.6...","[-4.425183874019593, -1.5259254737998595, -0.9...","[1953.1846064638203, 1953.1846064638203, 1953....",3.0
2,20000,"[-33.26517532883694, -26.39851069673757, -21.5...","[1.8311105685598315, -5.493331705679495, -5.64...","[1953.1846064638203, 1953.1846064638203, 1953....",3.0
3,30000,"[0.15259254737998595, 15.717032380138553, 34.9...","[0.45777764213995786, -1.5259254737998595, -2....","[1953.1846064638203, 1953.1846064638203, 1953....",3.0
4,40000,"[14.34369945371868, 12.512588885158848, 9.1555...","[1.2207403790398876, -1.5259254737998595, -1.2...","[1953.1846064638203, 1953.1846064638203, 1953....",3.0
